<a href="https://colab.research.google.com/github/Higgins2718/DS-Unit-2-Sprint-3-Classification-Validation/blob/master/DS_Unit_2_Sprint_Challenge_3_Classification_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 _Lambda School Data Science Unit 2_
 
 # Classification & Validation Sprint Challenge

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

#### For this Sprint Challenge, you'll predict whether a person's income exceeds $50k/yr, based on census data.

You can read more about the Adult Census Income dataset at the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/adult

#### Run this cell to load the data:

In [0]:
import pandas as pd

columns = ['age', 
           'workclass', 
           'fnlwgt', 
           'education', 
           'education-num', 
           'marital-status', 
           'occupation', 
           'relationship', 
           'race', 
           'sex', 
           'capital-gain', 
           'capital-loss', 
           'hours-per-week', 
           'native-country', 
           'income']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                 header=None, names=columns)

df['income'] = df['income'].str.strip()

## Part 1 — Begin with baselines

Split the data into an **X matrix** (all the features) and **y vector** (the target).

(You _don't_ need to split the data into train and test sets here. You'll be asked to do that at the _end_ of Part 1.)

In [0]:

X = df.drop(columns='income')

y = df['income']

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You can answer this question either with a scikit-learn function or with a pandas function.)

In [0]:
#y.value_counts(normalize=True)

majority = y.mode()[0]

y_pred = [majority] * len(y)

#y = np.where(df["income"].contains(">"), 1, other=0)

In [86]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

0.7591904425539756

What **ROC AUC score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of ROC AUC.)

With a majority class baseline, the ROC AUC score will be 0.5

In this Sprint Challenge, you will use **"Cross-Validation with Independent Test Set"** for your model validaton method.

First, **split the data into `X_train, X_test, y_train, y_test`**. You can include 80% of the data in the train set, and hold out 20% for the test set.

In [87]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [72]:
#X = X.drop("native-country", axis=1)

KeyError: ignored

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20)

In [89]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26048, 14), (6513, 14), (26048,), (6513,))

## Part 2 — Modeling with Logistic Regression!

- You may do exploratory data analysis and visualization, but it is not required.
- You may **use all the features, or select any features** of your choice, as long as you select at least one numeric feature and one categorical feature.
- **Scale your numeric features**, using any scikit-learn [Scaler](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of your choice.
- **Encode your categorical features**. You may use any encoding (One-Hot, Ordinal, etc) and any library (category_encoders, scikit-learn, pandas, etc) of your choice.
- You may choose to use a pipeline, but it is not required.
- Use a **Logistic Regression** model.
- Use scikit-learn's [**cross_val_score**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function. For [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules), use **accuracy**.
- **Print your model's cross-validation accuracy score.**

In [56]:
!pip install category_encoders

     |████████████████████████████████| 92kB 3.5MB/s 


In [0]:
%matplotlib inline
import warnings
import category_encoders as ce
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [0]:
X_train = X_train.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [93]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
26832,51,Private,205884,Some-college,10,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,40,United-States
19707,66,Self-emp-inc,112376,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States
29836,60,Local-gov,48788,Bachelors,13,Separated,Prof-specialty,Unmarried,White,Female,5455,0,55,United-States
31903,35,Local-gov,116960,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1887,40,United-States
24032,30,Self-emp-not-inc,79303,Assoc-voc,11,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,60,United-States


In [112]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

X_train["workclass"] = lb_make.fit_transform(X_train["workclass"])
X_train["education"] = lb_make.fit_transform(X_train["education"])
X_train["marital-status"] = lb_make.fit_transform(X_train["marital-status"])
X_train["occupation"] = lb_make.fit_transform(X_train["occupation"])
X_train["relationship"] = lb_make.fit_transform(X_train["relationship"])
X_train["race"] = lb_make.fit_transform(X_train["race"])
X_train["sex"] = lb_make.fit_transform(X_train["sex"])
X_train["native-country"] = lb_make.fit_transform(X_train["native-country"])


X_test["workclass"] = lb_make.fit_transform(X_test["workclass"])
X_test["education"] = lb_make.fit_transform(X_test["education"])
X_test["marital-status"] = lb_make.fit_transform(X_test["marital-status"])
X_test["occupation"] = lb_make.fit_transform(X_test["occupation"])
X_test["relationship"] = lb_make.fit_transform(X_test["relationship"])
X_test["race"] = lb_make.fit_transform(X_test["race"])
X_test["sex"] = lb_make.fit_transform(X_test["sex"])
X_test["native-country"] = lb_make.fit_transform(X_test["native-country"])


X["workclass"] = lb_make.fit_transform(X["workclass"])
X["education"] = lb_make.fit_transform(X["education"])
X["marital-status"] = lb_make.fit_transform(X["marital-status"])
X["occupation"] = lb_make.fit_transform(X["occupation"])
X["relationship"] = lb_make.fit_transform(X["relationship"])
X["race"] = lb_make.fit_transform(X["race"])
X["sex"] = lb_make.fit_transform(X["sex"])
X["native-country"] = lb_make.fit_transform(X["native-country"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [106]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
26832,51,4,205884,15,10,2,13,5,4,0,0,0,40,38
19707,66,5,112376,5,4,2,3,0,4,1,0,0,40,38
29836,60,2,48788,9,13,5,10,4,4,0,5455,0,55,38
31903,35,2,116960,12,14,2,10,0,4,1,0,1887,40,38
24032,30,6,79303,8,11,2,5,0,4,1,0,0,60,38


In [107]:
model = LogisticRegression(solver='lbfgs', multi_class='auto')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [108]:
model.score(X_test, y_test)

0.785659450330109

## Part 3 — Modeling with Tree Ensembles!

Part 3 is the same as Part 2, except this time, use a **Random Forest** or **Gradient Boosting** classifier. You may use scikit-learn, xgboost, or any other library. Then, print your model's cross-validation accuracy score.

In [0]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [120]:
from sklearn.model_selection import cross_val_score

# Trying both random forest and gradient boosting on the data... Let's see which is better
models = [ XGBClassifier(max_depth=1, n_estimators=100),
          RandomForestClassifier(max_depth=3, n_estimators=100, n_jobs=-1, random_state=42), 
          RandomForestClassifier(max_depth=None, n_estimators=100, n_jobs=-1, random_state=42)]

for model in models:
    print(model, '\n')
    score = cross_val_score(model, X, y, scoring='accuracy', cv=5).mean()
    print('Cross-Validation Accuracy:', score, '\n', '\n')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=1, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1) 

Cross-Validation Accuracy: 0.8496361982140425 
 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False) 

Cross-Validation Accuracy: 0.8196919806201242 
 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None

Best performing is the random forest model with no max_depth.

## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

In [267]:
cmat = [[ 85,  58], 
        [8, 36]]
tp = cmat[0][0] 
fp = cmat[0][1] 
fn = cmat[1][0] 
tn = cmat[1][1] 
# Precision and recall:tp/(tp+fp), tp/(tp+fn)
pd.DataFrame(cmat)


,0,1
0,85,58
1,8,36


Calculate accuracy

In [271]:
(tp+tn)/(tp+tn+fn+tn)

0.7333333333333333

Calculate precision

In [268]:
tp/(tp+fp)

0.5944055944055944

Calculate recall

In [269]:
tp/(tp+fn)

0.9139784946236559

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Experiment with feature selection, preprocessing, categorical encoding, and hyperparameter optimization, to try improving your cross-validation score.

### Part 3
Which model had the best cross-validation score? Refit this model on the train set and do a final evaluation on the held out test set — what is the test score? 

### Part 4
Calculate F1 score and False Positive Rate. 